In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from requests import Session
import json
import warnings
warnings.filterwarnings('ignore')
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime, timezone, timedelta

In [3]:
topn = 100

# api key dan url (will Delete Soon so replace to your own KEY)

api_key = '6f72fc32-b11d-496d-9f54-d4d4e5467042'  

#copy apikey mu di sini 

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

In [4]:
# setting parameter 
parameters = {
  'start':'1', # awal data coin yang diambil mulai dari rank 1
  'limit':'5000', # limit data coin diambil 5000 coin
  'convert':'USD' # pilih mata uang di sini kita pilih USD
}
headers = {
  'Accepts': 'application/json', # memilih format JSON
  'X-CMC_PRO_API_KEY': api_key, # memasukkan API Key
}


In [5]:
# code untuk call data
session = Session()
session.headers.update(headers)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    # code dibawah ini digunakan untuk mengambil data yang kemaren
    today = datetime.today()
    yesterday = str(today - timedelta(days = 1))
    yesterday_datetime = datetime.strptime(yesterday, "%Y-%m-%d %H:%M:%S.%f")
    for entry in data['data']:
        symbol = entry['symbol']
        # mengambil data date yang memiliki type data s
        date_added_str = entry['date_added'][:10]
        date_added = datetime.strptime(date_added_str, '%Y-%m-%d')
        if yesterday_datetime < date_added:
            print(symbol+': '+ date_added_str)
        else:
            pass
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

FGPT: 2023-06-14


In [6]:
# save data ke json
with open('crypto_apikey.json', 'w') as f:
    json.dump(data, f)
    print("Data tersimpan")

Data tersimpan


In [7]:
# membuat data sebagai dataframe
crypto_data = pd.DataFrame(data['data'])

# ambil data 'name','symbol','cmc_rank','last_updated','max_supply','quote' (quote untuk ambil price)
getting_price = crypto_data[['name','symbol','cmc_rank','last_updated','max_supply','quote']]

# mengambil price
getting_price['price'] = [getting_price['quote'].values[i]['USD'].get('price') for i in range(len(getting_price))]
datas = getting_price[['name','symbol','cmc_rank','max_supply','price','last_updated']].sort_values('price', ascending = False)
datas = datas.sort_values('cmc_rank')
top100 = datas[datas.cmc_rank <= topn]

# processing date data
top100['updated_date'] = pd.to_datetime(top100['last_updated']).dt.date
top100['updated_time'] = pd.to_datetime(top100['last_updated']).dt.time

top100['updated_time'] = pd.to_timedelta(top100['updated_time'].astype('str')) + timedelta(hours=7, minutes=0, seconds=0)
top100['updated_time'] = pd.to_datetime(top100['updated_time'].astype('str').str.replace('0 days','').str.strip()).dt.time

print(top100)

              name symbol  cmc_rank    max_supply         price  \
0          Bitcoin    BTC         1  2.100000e+07  25994.750162   
1         Ethereum    ETH         2           NaN   1744.539257   
2           Tether   USDT         3           NaN      0.999817   
3              BNB    BNB         4           NaN    251.064680   
4         USD Coin   USDC         5           NaN      0.999964   
..             ...    ...       ...           ...           ...   
95          Gnosis    GNO        96  3.000000e+06    109.104822   
96     WOO Network    WOO        97           NaN      0.165078   
97         Zilliqa    ZIL        98  2.100000e+10      0.017594   
98        MX TOKEN     MX        99           NaN      2.734065   
99  Convex Finance    CVX       100  1.000000e+08      3.463277   

                last_updated updated_date updated_time  
0   2023-06-14T04:31:00.000Z   2023-06-14     11:31:00  
1   2023-06-14T04:31:00.000Z   2023-06-14     11:31:00  
2   2023-06-14T04:31:00.